In [27]:
import warnings

# 오류 경고 무시하기
warnings.filterwarnings(action='ignore')

# 1. 데이터 불러오기

In [28]:
import pandas as pd

redefined_df = pd.read_csv("../input_csv/redefined_df.csv")

# 서비스 업종별 데이터프레임을 딕셔너리 형태로 저장
service_type_list = list(redefined_df.서비스_업종_코드_명.unique())
service_type_dictionary = {}

for serviece in service_type_list:
    service_type_dictionary[f"{serviece}"] = redefined_df.loc[redefined_df['서비스_업종_코드_명'] == serviece]

# 2. 지표 관련 처리

In [31]:
# 서비스 업종별 데이터프레임의 군집 지표 점수 변수를 생성
cluster_index_dict = {}

for key, value in service_type_dictionary.items():
    cluster_index_dict[f'{key}'] = value.groupby(by=['서비스_업종_코드_명', 'cluster'])[['집객력(점수)', '매출액(점수)', '경쟁업체(점수)', '편의성(점수)', '잠재고객(점수)']]\
                                        .mean()\
                                        .reset_index()

# 군집 지표 점수를 바탕으로 각 군집의 등급 변수를 생성
for key, value in cluster_index_dict.items():
    cluster_index_dict[key]['평균'] = (cluster_index_dict[key]['집객력(점수)'] + cluster_index_dict[key]['매출액(점수)'] + cluster_index_dict[key]['경쟁업체(점수)'] + cluster_index_dict[key]['편의성(점수)'] + cluster_index_dict[key]['잠재고객(점수)'])/5
    for i in range(len(cluster_index_dict[key])):
        if cluster_index_dict[key].loc[i,'평균'] < 2:
            cluster_index_dict[key].loc[i,'등급'] = 'D'
        elif cluster_index_dict[key].loc[i,'평균'] < 3:
            cluster_index_dict[key].loc[i,'등급'] = 'C'
        elif cluster_index_dict[key].loc[i,'평균'] < 4:
            cluster_index_dict[key].loc[i,'등급'] = 'B'
        else :
            cluster_index_dict[key].loc[i,'등급'] = 'A'

# 업종별로 나뉘어져 있던 데이터프레임을 하나의 데이터프레임으로 결합
cluster_charicteristics_df = pd.concat(cluster_index_dict.values(), ignore_index = True).rename(columns = {"집객력(점수)" : "집객력",
                                                                                         "매출액(점수)" : "매출액",
                                                                                         "경쟁업체(점수)" : "경쟁업체",
                                                                                         "편의성(점수)" : "편의성",
                                                                                         "잠재고객(점수)" : "잠재고객",})

cluster_charicteristics_df

cluster_charicteristics_df.to_csv("../input_csv/cluster_charicteristics_df.csv", index = False)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 24)

In [ ]:
cluster_charicteristics_df